In [1]:
from __future__ import absolute_import, division, print_function

import os
import argparse
import fnmatch
import glob
import cv2
import numpy as np
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [2]:
def compute_errors(gt, pred):
    thresh = np.maximum((gt / pred), (pred / gt))
    d1 = (thresh < 1.25).mean()
    d2 = (thresh < 1.25 ** 2).mean()
    d3 = (thresh < 1.25 ** 3).mean()

    rmse = (gt - pred) ** 2
    rmse = np.sqrt(rmse.mean())

    rmse_log = (np.log(gt) - np.log(pred)) ** 2
    rmse_log = np.sqrt(rmse_log.mean())

    abs_rel = np.mean(np.abs(gt - pred) / gt)

    err = np.abs(np.log10(pred) - np.log10(gt))
    log10 = np.mean(err)

    return  log10, abs_rel, rmse, rmse_log, d1, d2, d3

In [3]:
def eval(preds,gts):

    num_samples = len(preds)

    log10 = np.zeros(num_samples, np.float32)
    rms = np.zeros(num_samples, np.float32)
    log_rms = np.zeros(num_samples, np.float32)
    abs_rel = np.zeros(num_samples, np.float32)
    d1 = np.zeros(num_samples, np.float32)
    d2 = np.zeros(num_samples, np.float32)
    d3 = np.zeros(num_samples, np.float32)
    
    for i in range(num_samples):

        gt= gts[i]
        pred = preds[i]

        min_depth_eval = 1e-8
        max_depth_eval = 255
        pred[pred < min_depth_eval] = 1e-3
        pred[pred > max_depth_eval] = max_depth_eval
        gt[gt < min_depth_eval] = 1e-3
        gt[gt > max_depth_eval] = max_depth_eval


        log10[i], abs_rel[i], rms[i], log_rms[i], d1[i], d2[i], d3[i] = compute_errors(gt, pred)

    print("{:>7}, {:>7}, {:>7}, {:>7}, {:>7}, {:>7}, {:>7}".format(
        'd1', 'd2', 'd3', 'AbsRel',  'RMSE', 'RMSElog',  'log10'))
    print("{:7.3f}, {:7.3f}, {:7.3f}, {:7.3f}, {:7.3f}, {:7.3f}, {:7.3f}".format(
        d1.mean(), d2.mean(), d3.mean(),
        abs_rel.mean(), rms.mean(), log_rms.mean(), log10.mean()))

    return log10, abs_rel, rms, log_rms, d1, d2, d3

In [4]:
def test():
    gts = []
    preds = []
    pred_path = 'output/depths/'
    gt_path = 'gt/depths/'
    # Use glob to get a list of file paths for all JPG files in the folder
    pred_files = glob.glob(pred_path + '*.jpg')
    for file_path in pred_files:
        pred = cv2.imread(file_path, -1)
        pred = pred / 256.0
        preds.append(pred)

    print('Raw jpg files reading done')
    print('Evaluating {} files'.format(len(preds)))

    gt_files = glob.glob(gt_path + '*.jpg')
    for file_path in gt_files:
        gt = cv2.imread(file_path, -1)
        gt = gt.mean(axis = 2)/ 256
        gts.append(gt)
    print('GT files reading done')
    print('Evaluating {} gt files'.format(len(gts)))

    print('Computing errors')
    eval(preds,gts)

    print('Done.')

In [5]:
pred = 255 - cv2.imread('121.jpg',-1)
pred = pred /256
gt = cv2.imread('121_gt.jpg',-1)
gt = gt.mean(axis = 2)/ 256

min_depth_eval = 1e-8
max_depth_eval = 255
pred[pred < min_depth_eval] = 1e-3
pred[pred > max_depth_eval] = max_depth_eval
gt[gt < min_depth_eval] = 1e-3
gt[gt > max_depth_eval] = max_depth_eval


In [6]:
test()

Raw jpg files reading done
Evaluating 101 files
GT files reading done
Evaluating 101 gt files
Computing errors


UnboundLocalError: local variable 'gt' referenced before assignment

In [ ]:
# cv2.imshow('pred',pred)
# cv2.imshow('gt',gt)
# cv2.waitKey(0) 
# # closing all open windows 
# cv2.destroyAllWindows() 